In [ ]:
#pip install pyfinite

In [ ]:
#importing req moduless
import numpy as np
from pyfinite import ffield

In [ ]:
#to perform field operations
F = ffield.FField(7)

In [ ]:
#function to convert inputs or outputs  into int values
def int_conv(string):
    #to store converted values
    converted = []
    #taking two chars at a time
    for j in range(0, 16, 2):
        #getting char to binary
        b_1 = bin(ord(string[j]) - 102)[2:].zfill(4)
        b_2 = bin(ord(string[j + 1]) - 102)[2:].zfill(4)
        #getting the int value of the current byte
        byte_int = int(b_1 + b_2, 2)
        #appending it to converted list
        converted.append(byte_int)

    return converted

In [ ]:
#function to perform Exponent operation on an element
def exponent(byte_int,expo_value):
    #to store the exponent value
    val = 1
    #performing the exponent operation
    for i in range(expo_value):
        val = F.Multiply(val,byte_int)
    #returning the value
    return val

In [ ]:
#function to perform E operation
def E_OP(N,E):
    #performing E operation
    for i in range(8):
        N[i] = exponent(N[i], E[i])
    return N

In [ ]:
#function to perform A operation
def A_OP(N,A):
    #to store output of operation
    temp=[0] * 8
    #peforming A operation
    for i in range(8):
        for j in range(8):
            temp[i] = temp[i] ^ F.Multiply(A[i][j], N[j])
    return temp

In [ ]:
#function to perform EAEAE operations
def EAEAE(A,E,N):
    #performing E operation
    N=E_OP(N,E)
    #performing A operation
    N=A_OP(N,A)
    #performing E operation
    N=E_OP(N,E)
    #performing A operation
    N=A_OP(N,A)
    #performing E operation
    N=E_OP(N,E)
    
    return N

In [ ]:
#opening plain and cipher text files
input_file= open('plaintexts.txt','r')
output_file= open('output.txt','r')
#reading plaintexts and ciphertexts
inputs = input_file.readlines()
outputs = output_file.readlines()

#to store possible diagonal values of A and elements of E pairs
possible_pairs = [[],[],[],[],[],[],[],[]]

# generating possible pairs of E and A(diagonal elements) for every byte value
for i in range(8):
    #getting current input
    curr_input = inputs[i]
    #converting curr input into integers
    input_byte_ints=int_conv(curr_input)
    #getting current output
    curr_output = outputs[i]
    #converting curr output into integers
    output_byte_ints=int_conv(curr_output)
   
    #lopping through possible e and a values
    for e in range(1,127):
        for a in range(0,128):
            #performing EAEAE on single byte(the non zero byte)
            curr = exponent(input_byte_ints[i],e)
            curr = F.Multiply(curr,a)
            curr = exponent(curr,e)
            curr = F.Multiply(curr,a)
            curr = exponent(curr,e)
            
            #if curr value is equal to corresponding cipher byte value ,
            #then e,a are possible pair
            if curr==output_byte_ints[i]:
                pair = []
                pair.append(a)
                pair.append(e)
                #appending the current pair to possible pairs
                possible_pairs[i].append(pair)

In [ ]:
#further filtering of possible pairs based on other inputs and outputs
for i in range(8,len(inputs)):
    #getting the current byte no in input that is non zero
    ind = i % 8
    #getting current input
    curr_input = inputs[i]
    #converting curr input into integers
    input_byte_ints=int_conv(curr_input)
    #getting current output
    curr_output = outputs[i]
    #converting curr output into integers
    output_byte_ints=int_conv(curr_output)

    #to store filtered pairs
    filtered_pair= []

    for a,e in possible_pairs[ind]:
        #performing EAEAE on single byte(the non zero byte)
        curr = exponent(input_byte_ints[ind],e)
        curr = F.Multiply(curr,a)
        curr = exponent(curr,e)
        curr = F.Multiply(curr,a)
        curr = exponent(curr,e)

        #if curr value is equal to corresponding cipher byte value ,
        #then e,a are possible pair
        if curr==output_byte_ints[ind]:
            pair = []
            pair.append(a)
            pair.append(e)
            #appending to filtered pair
            filtered_pair.append(pair)
    #assigning filted pair
    possible_pairs[ind] = filtered_pair

In [ ]:
possible_pairs

In [ ]:
#to store the final pairs of E and A(diagonal elements) 
final_pairs = [[],[],[],[],[],[],[],[]]
#to store the elements below the diagonal elements of A(since last diagonal element does not have any element below it)
#we need a list of only 7 elements
diag_below = [0]*7

#possible below diagonal indices
for i in range(7):
    #for each possible pairs of current diagonal element and its next diagonal element
    for a1,e1 in possible_pairs[i]:
        for a2,e2 in possible_pairs[i+1]:
            #for possible diag_below element values
            for poss_dbi in range(0,128):
                f=0
                for traverse in range(127):
                    #getting the curr input
                    curr_input = inputs[traverse*8+i]
                    #converting curr input into integers
                    input_byte_ints=int_conv(curr_input)
                    #getting curr output
                    curr_output = outputs[traverse*8+i]
                    #converting curr output into integers
                    output_byte_ints=int_conv(curr_output)
                    
                    #getting the cipher byte value of the input byte next of non zero byte
                    first_e= exponent(input_byte_ints[i],e1)
                    curr_byte = F.Multiply(first_e, a1)
                    next_byte = F.Multiply(first_e, poss_dbi)

                    curr_byte_e = exponent(curr_byte,e1)
                    next_byte_e = exponent(next_byte,e2)

                    curr_byte = F.Multiply(curr_byte_e, a1)
                    next_byte = F.Multiply(curr_byte_e, poss_dbi) ^ F.Multiply(next_byte_e, a2)

                    curr_byte = exponent(curr_byte,e1)
                    next_byte = exponent(next_byte,e2)
                    
                    #if that curr cipher byte is not equal to actual cipher byte
                    if next_byte != output_byte_ints[i + 1]:
                        #breaking
                        f=1
                        break
                #if f==0 then e1,a1 and e2,a2 are actual pairs
                if f==0:
                    final_pairs[i]=[a1,e1]
                    final_pairs[i+1]=[a2,e2]
                    diag_below[i]=poss_dbi

In [ ]:
#to store 'A' and 'E'
A = [[0]*8,[0]*8,[0]*8,[0]*8,[0]*8,[0]*8,[0]*8,[0]*8]
E = [0,0,0,0,0,0,0,0]

# adding diagonal elements and below diagonal elementsof A and elements of E
for i in range(len(final_pairs)):
    A[i][i] = final_pairs[i][0]
    E[i] = final_pairs[i][1]
    if i>0:
        A[i][i-1] = diag_below[i-1]

In [ ]:
#GETTING THE OTHER NON DIAGONAL ELEMENTS

#i,k,j are used to loop through the indexes of unfilled non diagonal elements
for i in range(2,8):
    k=i
    for j in range(0,8-i):
        
        #looping through all the possible values of that element
        for curr_poss in range(0,128):
            f=0
            #used for looping through inputs
            for traverse in range(127):
                #getting curr input
                curr_input = inputs[traverse * 8 + j]
                #converting curr input into integers
                input_byte_ints=int_conv(curr_input)
                #getting curr output
                curr_output = outputs[traverse * 8 + j]
                #converting curr output into integers
                output_byte_ints=int_conv(curr_output)
                #assigning the possible value
                A[k][j] = curr_poss
                #performing EAEAE
                curr_output_byte_ints= EAEAE(A,E,input_byte_ints)
                #if that curr cipher byte is not equal to actual cipher byte
                if curr_output_byte_ints[k] != output_byte_ints[k]:
                    #BREAKING
                    f = 1
                    break
            #if f==0 then curr_poss is the  actual value,so breaking
            if f == 0:
                break
        k+=1

In [ ]:
print("A = ")
for a in A:
    print(a,"\n")

print("E = ")
print(E)